In [5]:
import numpy as np

# Gaussian Elimination

## LU factorization

In [6]:
A = np.array([[2,4,-2],[4,9,-3],[-2,-3,7]])
A

array([[ 2,  4, -2],
       [ 4,  9, -3],
       [-2, -3,  7]])

In [8]:
def my_lu_3loops(A):
    'this function factors A into lower L x upper U. It uses 3 nested for-loops'
    
    # size of A
    m,n=A.shape
    # check if A is square
    if m!=n:
        print('WARNING: A must be a square matrix')
        return
    
    # initialize U and L factors
    L = np.eye(n) # nxn identity matrix
    U = A.copy()
    
    # Gaussian Elimination steps
    for i in range(n):
        # loop over rows i+1,...,n
        for j in range(i+1,n):
            # compute the multiplier
            L[j,i] = U[j,i]/U[i,i]
            # update U
            U[j,i] = 0
            for k in range(i+1,n):
                U[j,k] = U[j,k] - L[j,i]*U[i,k]
    return L,U

In [9]:
# check that the function works
L,U = my_lu_3loops(A)

In [10]:
L

array([[ 1.,  0.,  0.],
       [ 2.,  1.,  0.],
       [-1.,  1.,  1.]])

In [11]:
U

array([[ 2,  4, -2],
       [ 0,  1,  1],
       [ 0,  0,  4]])

In [12]:
# check that L times U equals A
L.dot(U) - A

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [13]:
def my_lu_2loops(A):
    'this function factors A into lower L x upper U. It uses 2 nested for-loops'
    
    # size of A
    m,n=A.shape
    # check if A is square
    if m!=n:
        print('WARNING: A must be a square matrix')
        return
    
    # initialize U and L factors
    L = np.eye(n) # nxn identity matrix
    U = A.copy()
    
    # Gaussian Elimination steps
    for i in range(n):
        # loop over rows i+1,...,n
        for j in range(i+1,n):
            # compute the multiplier
            L[j,i] = U[j,i]/U[i,i]
            # update U
            U[j,i] = 0
            U[j,i+1:n] = U[j,i+1:n] - L[j,i]*U[i,i+1:n]
    return L,U

In [14]:
# check that the function works
L,U = my_lu_2loops(A)

In [15]:
L

array([[ 1.,  0.,  0.],
       [ 2.,  1.,  0.],
       [-1.,  1.,  1.]])

In [16]:
U

array([[ 2,  4, -2],
       [ 0,  1,  1],
       [ 0,  0,  4]])

In [17]:
# check that L times U equals A
L.dot(U) - A

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [18]:
def my_lu(A,pivoting=None):
    'this function factors A into lower L x upper U. It uses 1 for-loop'
    
    n,m = A.shape # size of matrix A
    if m!=n: # check if A is square
        print('WARNING: A must be a square matrix') # print a warning massage
        return  
    
    # initialize U and L factors
    U = A.copy()
    L = np.identity(n)
    
    
    # Gaussian Elimination steps
    for i in range(n): 
        # multipliers 
        L[i+1:n,i] = U[i+1:n,i]/U[i,i] 
        # update U
        U[i+1:n,i] = 0 
        U[i+1:n,i+1:n] = U[i+1:n,i+1:n] - np.outer(L[i+1:n,i],U[i,i+1:n]) # update U

    return L,U

In [19]:
# check that the function works
L,U = my_lu(A)

In [20]:
L

array([[ 1.,  0.,  0.],
       [ 2.,  1.,  0.],
       [-1.,  1.,  1.]])

In [21]:
U

array([[ 2,  4, -2],
       [ 0,  1,  1],
       [ 0,  0,  4]])

In [22]:
# check that L times U equals A
L.dot(U) - A

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

Let us time our three LU functions

In [26]:
# n x n random matrix
n = 500
A = np.random.randn(n,n)

In [27]:
%%timeit
my_lu_3loops(A)

36.1 s ± 307 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%%timeit
my_lu_2loops(A)

981 ms ± 16.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%%timeit
my_lu(A)

343 ms ± 16.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Solving linear systems Ax=b by using the LU factorization

In [30]:
def my_solve(A,b):
    'this function solves Ax=b'

    # size of the problem
    n = len(b)
    
    # initialize vectors x and y
    y = np.zeros(n)
    x = np.zeros(n)

    # compute LU factorization
    L,U = my_lu(A)

    # solve Ly = b
    for i in range(n):
        y[i] = b[i] - L[i,0:i].dot(y[0:i])

    # solve Ux = y
    for i in reversed(range(n)): # i = n,...,1
        x[i] = (y[i] - U[i,i+1:n].dot(x[i+1:n]))/U[i,i]

    return x

In [31]:
def my_solve_lu(L,U,b):
    'this function solves Ax=b given the LU factorization of A'

    # size of the problem
    n = len(b)
    
    # initialize vectors x and y
    y = np.zeros(n)
    x = np.zeros(n)


    # solve Ly = b
    for i in range(n):
        y[i] = b[i] - L[i,0:i].dot(y[0:i])

    # solve Ux = y
    for i in reversed(range(n)): # i = n,...,1
        x[i] = (y[i] - U[i,i+1:n].dot(x[i+1:n]))/U[i,i]

    return x

In [38]:
# small 3 x 3 random linear system
A = np.random.randn(3,3)
b = np.random.randn(3)

In [39]:
my_solve(A,b)

array([-2.92336184,  6.59144829, -7.89428344])

In [42]:
L,U = my_lu(A)
my_solve_lu(L,U,b)

array([-2.92336184,  6.59144829, -7.89428344])

In [43]:
# NumPy built-in function
np.linalg.solve(A,b)

array([-2.92336184,  6.59144829, -7.89428344])

## Same matrix A, multiple right-hand side vectors

In [44]:
# matrix A
A = np.random.randn(500,500)
# 10 right-hand side vectors
B = np.random.randn(500,10)

In [45]:
%%timeit
for i in range(10):
    my_solve(A,B[:,i])

3.3 s ± 50.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [46]:
%%timeit
L,U = my_lu(A) # we compute LU factorization once
for i in range(10):
    my_solve_lu(L,U,B[:,i]) # then, we use the LU factorization to solve the linear systems

354 ms ± 5.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Numerical instability example

In [47]:
# the matrix A has a small pivot
e = 1e-16 
A = np.array([[e,1],[1,1]])
A

array([[1.e-16, 1.e+00],
       [1.e+00, 1.e+00]])

In [48]:
# the computed L and U factors are close to the exact ones (we computed those by hand in class)
L, U = my_lu(A)

In [49]:
L

array([[1.e+00, 0.e+00],
       [1.e+16, 1.e+00]])

In [50]:
U

array([[ 1.e-16,  1.e+00],
       [ 0.e+00, -1.e+16]])

In [52]:
# but L times U is not close to A !!!!
L.dot(U) - A

array([[ 0.,  0.],
       [ 0., -1.]])

In [53]:
# if we try to solve a linear system with the computed L and U factors...
b = np.array([1,0])

In [55]:
# by using our function
my_solve_lu(L,U,b)

array([0., 1.])

In [56]:
# by using NumPy built-in function
np.linalg.solve(A,b)

array([-1.,  1.])

In [57]:
# which solution is correct?? 

## Gaussian Elimination with Partial Pivoting

We can use vectors to store permutation matrices

In [58]:
A = np.arange(16).reshape(4,4)
A

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [59]:
p = [3,2,1,0] # row order
A[p]

array([[12, 13, 14, 15],
       [ 8,  9, 10, 11],
       [ 4,  5,  6,  7],
       [ 0,  1,  2,  3]])

In [60]:
p = [2,1,3,0]
A[p]

array([[ 8,  9, 10, 11],
       [ 4,  5,  6,  7],
       [12, 13, 14, 15],
       [ 0,  1,  2,  3]])

In [64]:
def my_lu(A,pivoting=None):
    
    """
    This function factors A = LU (if pivoting=None; default) or PA = LU (if pivoting='partial')
    INPUT:
        matrix A
    OUTPUT:
        permutation vector p (if pivoting == 'partial')
        lower triangular factor L
        upper triangular factor U
    """
    
    n,m = A.shape # size of matrix A
    if m!=n: # check if A is square
        print('WARNING: A must be a square matrix') # print a warning massage
        return  
    
    # initialize factors L and U
    U = A.copy()
    L = np.identity(n)
    
    # initialize permutation vector p = [0,1,...,n-1]
    if pivoting == 'partial':
        p = np.arange(n)
    
    # GAUSSIAN ELIMINATION 
    for i in range(n):  # loop over columns 1,...,n (first loop)
        if pivoting=='partial':
            # Find the entry with largest absolute value
            row2swap = i + np.argmax(np.abs(U[i:n,i]))
            # swap rows of U
            U[[row2swap,i],i:n] = U[[i,row2swap],i:n]
            # swap multipliers
            L[[row2swap,i],0:i] = L[[i,row2swap],0:i] 
            # update permutation vector p
            p[[row2swap,i]]=p[[i,row2swap]]
        # multipliers 
        L[i+1:n,i] = U[i+1:n,i]/U[i,i] 
        # update U
        U[i+1:n,i] = 0 
        U[i+1:n,i+1:n] = U[i+1:n,i+1:n] - np.outer(L[i+1:n,i],U[i,i+1:n]) # update U
    
    if pivoting=='partial':
        return L,U,p
    else:
        return L,U

In [75]:
A = np.array([[2,4,-2],[4,9,-3],[-2,-3,7]])
A = A.astype('float')
A

array([[ 2.,  4., -2.],
       [ 4.,  9., -3.],
       [-2., -3.,  7.]])

In [80]:
L,U,p = my_lu(A,pivoting='partial')

In [81]:
L

array([[ 1.        ,  0.        ,  0.        ],
       [-0.5       ,  1.        ,  0.        ],
       [ 0.5       , -0.33333333,  1.        ]])

In [82]:
U

array([[ 4.        ,  9.        , -3.        ],
       [ 0.        ,  1.5       ,  5.5       ],
       [ 0.        ,  0.        ,  1.33333333]])

In [83]:
# check that PA = LU
A[p] - L.dot(U)

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -2.22044605e-16]])